In [3]:
import json
from os.path import basename, join
import os 
import shutil
from os import makedirs

from smart_open import open as smart_open
from dask.distributed import Client
import xarray as xr
import fsspec
from rechunker import rechunk

%matplotlib inline

def get_json(uri):
    with smart_open(uri) as fd:
        return json.load(fd)

client = Client()

distributed.diskutils - INFO - Found stale lock file and directory '/opt/src/notebooks/dask-worker-space/worker-3niqvzlv', purging
distributed.diskutils - INFO - Found stale lock file and directory '/opt/src/notebooks/dask-worker-space/worker-e8ikn7g2', purging
distributed.diskutils - INFO - Found stale lock file and directory '/opt/src/notebooks/dask-worker-space/worker-i5xh8wij', purging


In [3]:
nwm_uri = 's3://noaa-nwm-retrospective-2-1-zarr-pds/chrtout.zarr'
ds = xr.open_zarr(fsspec.get_mapper(nwm_uri, anon=True, requester_pays=True))

In [12]:
out_uri = 's3://azavea-noaa-hydro-data/noaa/huc2-comids.json'
comids = get_json(out_uri)['comids']
time_range = slice('01-01-1990', '01-02-1990')
# Apparently, only some of the comids are available in NWM.
avail_comids = list(set(ds.feature_id.values).intersection(set(comids)))
avail_comids.sort()
print(
    f'There are {len(comids)} reaches in the HUC and {len(avail_comids)} of those are in NWM.')

There are 137288 reaches in the HUC and 122256 of those are in NWM.


In [32]:
sub_ds = ds.sel(time=time_range, feature_id=avail_comids)

# Fails without this block. See https://github.com/pydata/xarray/issues/5219 
def remove_chunks_encoding(ds):
    for var in list(ds.keys()) + list(ds.coords):
        if 'chunks' in ds[var].encoding:
            del ds[var].encoding['chunks']
    return ds

sub_ds = remove_chunks_encoding(sub_ds)
sub_ds

<xarray.Dataset>
Dimensions:     (feature_id: 122256, time: 48)
Coordinates:
    elevation   (feature_id) float32 dask.array<chunksize=(122256,), meta=np.ndarray>
  * feature_id  (feature_id) int32 1748535 1748537 ... 932080037 932080038
    gage_id     (feature_id) |S15 dask.array<chunksize=(122256,), meta=np.ndarray>
    latitude    (feature_id) float32 dask.array<chunksize=(122256,), meta=np.ndarray>
    longitude   (feature_id) float32 dask.array<chunksize=(122256,), meta=np.ndarray>
    order       (feature_id) int32 dask.array<chunksize=(122256,), meta=np.ndarray>
  * time        (time) datetime64[ns] 1990-01-01 ... 1990-01-02T23:00:00
Data variables:
    crs         |S1 ...
    streamflow  (time, feature_id) float64 dask.array<chunksize=(48, 445), meta=np.ndarray>
    velocity    (time, feature_id) float64 dask.array<chunksize=(48, 445), meta=np.ndarray>
Attributes:
    TITLE:                OUTPUT FROM WRF-Hydro v5.2.0-beta2
    code_version:         v5.2.0-beta2
    featureType:          timeSeries
    model_configuration:  retrospective
    proj4:                +proj=lcc +units=m +a=6370000.0 +b=6370000.0 +lat_1...

In [33]:
sub_ds.streamflow

<xarray.DataArray 'streamflow' (time: 48, feature_id: 122256)>
dask.array<getitem, shape=(48, 122256), dtype=float64, chunksize=(48, 24143), chunktype=numpy.ndarray>
Coordinates:
    elevation   (feature_id) float32 dask.array<chunksize=(122256,), meta=np.ndarray>
  * feature_id  (feature_id) int32 1748535 1748537 ... 932080037 932080038
    gage_id     (feature_id) |S15 dask.array<chunksize=(122256,), meta=np.ndarray>
    latitude    (feature_id) float32 dask.array<chunksize=(122256,), meta=np.ndarray>
    longitude   (feature_id) float32 dask.array<chunksize=(122256,), meta=np.ndarray>
    order       (feature_id) int32 dask.array<chunksize=(122256,), meta=np.ndarray>
  * time        (time) datetime64[ns] 1990-01-01 ... 1990-01-02T23:00:00
Attributes:
    grid_mapping:  crs
    long_name:     River Flow
    units:         m3 s-1

In [34]:
sub_ds = remove_chunks_encoding(sub_ds.chunk(1000))

In [35]:
sub_ds.streamflow

<xarray.DataArray 'streamflow' (time: 48, feature_id: 122256)>
dask.array<rechunk-merge, shape=(48, 122256), dtype=float64, chunksize=(48, 1000), chunktype=numpy.ndarray>
Coordinates:
    elevation   (feature_id) float32 dask.array<chunksize=(1000,), meta=np.ndarray>
  * feature_id  (feature_id) int32 1748535 1748537 ... 932080037 932080038
    gage_id     (feature_id) |S15 dask.array<chunksize=(1000,), meta=np.ndarray>
    latitude    (feature_id) float32 dask.array<chunksize=(1000,), meta=np.ndarray>
    longitude   (feature_id) float32 dask.array<chunksize=(1000,), meta=np.ndarray>
    order       (feature_id) int32 dask.array<chunksize=(1000,), meta=np.ndarray>
  * time        (time) datetime64[ns] 1990-01-01 ... 1990-01-02T23:00:00
Attributes:
    grid_mapping:  crs
    long_name:     River Flow
    units:         m3 s-1

In [36]:
# save zarr to disk locally for testing purposes
root_dir = '/opt/data/noaa/rechunker-test'
if os.path.isdir(root_dir):
    shutil.rmtree(root_dir)
makedirs(root_dir)

target_uri = join(root_dir, 'chrtout-test-06-29-2021.zarr')

for var in sub_ds:
    if 'chunks' in sub_ds[var].encoding:
        del sub_ds[var].encoding['chunks']

sub_ds.to_zarr(target_uri)

In [38]:
sub_ds.chunks

Frozen({'feature_id': (1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 256), 'time': (48,)})

In [4]:
target_chunks = {'feature_id': 300, 'time': 300}
max_mem = '1GB'

root_dir = '/opt/data/noaa/rechunker-test/rechunk/'
if os.path.isdir(root_dir):
    shutil.rmtree(root_dir)
makedirs(root_dir)

test_uri = '/opt/data/noaa/rechunker-test/chrtout-test-06-29-2021.zarr'
ds = xr.open_zarr(test_uri)

target_store = join(root_dir, 'nwm-target.zarr')
temp_store = join(root_dir, 'nwm-temp.zarr')

# Note, if you get a ContainsArrayError, you probably need to delete temp_store and target_store first.
# See https://github.com/pangeo-data/rechunker/issues/78
rechunk_plan = rechunk(ds, target_chunks, max_mem, target_store, temp_store=temp_store)
rechunk_plan.execute()

<zarr.hierarchy.Group '/'>

In [5]:
rechunked_ds = xr.open_zarr(target_store)
rechunked_ds.chunks


/tmp/ipykernel_9347/3890022654.py:1: RuntimeWarning: Failed to open Zarr store with consolidated metadata, falling back to try reading non-consolidated metadata. This is typically much slower for opening a dataset. To silence this warning, consider:
1. Consolidating metadata in this existing store with zarr.consolidate_metadata().
2. Explicitly setting consolidated=False, to avoid trying to read consolidate metadata, or
3. Explicitly setting consolidated=True, to raise an error in this case instead of falling back to try reading non-consolidated metadata.
  rechunked_ds = xr.open_zarr(target_store)


Frozen({'feature_id': (300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 30